# BERT for BETO

In [8]:
import pandas as pd
import numpy as np
import torch
import random
import os
import nltk
from nltk import tokenize
from transformers import BertTokenizer

In [2]:
#Load in some data
save_path = '/Users/Jonathan/Desktop/LabeledChemEData/Labeled_Sheets/'
#Do we have something that allows us to fill empty cells with "" or something else? Maybe set any NaNs to 0? That would imply
#A null label. 
df = pd.read_excel(save_path+"Carbon_0.xlsx")

In [3]:
df

,Unnamed: 0,name,tokens,BESIO,entity,mol_class,Unnamed: 6,name.1,tokens.1,BESIO.1,...,tokens.48,BESIO.48,entity.48,mol_class.48,Unnamed: 294,name.49,tokens.49,BESIO.49,entity.49,mol_class.49
0,0,Jon O,In,NaN,NaN,NaN,0,Jon O,X-ray,NaN,...,©,NaN,NaN,NaN,0,Jon O,©,NaN,NaN,NaN
1,1,2010,the,NaN,NaN,NaN,1,2014,photoelectron,NaN,...,2020,NaN,NaN,NaN,1,2015,2015,NaN,NaN,NaN
2,2,250,interaction,NaN,NaN,NaN,2,114,spectroscopy,NaN,...,Elsevier,NaN,NaN,NaN,2,104,Elsevier,NaN,NaN,NaN
3,3,10.1016/j.carbon.2010.02.003,between,NaN,NaN,NaN,3,10.1016/j.carbon.2013.12.061,(XPS),NaN,...,Ltd,NaN,NaN,NaN,3,10.1016/j.carbon.2015.08.007,Ltd.,NaN,NaN,NaN
4,4,NaN,gas,NaN,NaN,NaN,4,NaN,has,NaN,...,The,NaN,NaN,NaN,4,NaN,Hydrogels,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,349,NaN,NaN,NaN,NaN,NaN,349,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,349,NaN,NaN,NaN,NaN,NaN
350,350,NaN,NaN,NaN,NaN,NaN,350,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,350,NaN,NaN,NaN,NaN,NaN
351,351,NaN,NaN,NaN,NaN,NaN,351,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,351,NaN,NaN,NaN,NaN,NaN
352,352,NaN,NaN,NaN,NaN,NaN,352,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,352,NaN,NaN,NaN,NaN,NaN


Need to update extract_xy_ to pull out just the x values (ie, tokens), and the y values as a smashed together BESIO. He used NLTK sent_tokenize, so we are in the same place, and we can just go back to having sentences or something.

In [7]:
def extract_xy(df):
    """
    This method extracts and correctly aranges the NER training x-values (tokens)
    and y-values (BESIO labels) from a pandas dataframe containing labeled NER
    data

    Parameters:
        df (pandas DataFrame, required): Dataframe loaded via pd.read_excel() on
            a labeled NER dataset

        endings_dict (dictionary, required): Dictionary containing the indicies
            where each sentence in each line of tokens ends.

    Returns:
        two lists of identical shape. One contains all the words that were tokenized for labeling
        and the other contains all the labels of those tokenized words.
    """
    labeled = []
    columns = df.columns
    new_df = pd.DataFrame()
    all_tokens = []
    besio = []
    mol = []
    IorO = []
        
    for idx, column in enumerate(columns):
        # find every column that starts with 'name'
        if column.startswith('name'):

            # check if the entry in 'name' cell is a str
            if isinstance(df[column][0], str):
                tokens = df[columns[idx + 1]].values
                #find the index where the tokens become NaNs, and chop the token length down to that size. 
                l = 0
                for entries in tokens: 
                    if type(entries) == str:
                        l += 1
                all_tokens.append(tokens[:l])
                df[columns[idx+2]].replace(np.nan, 'O', inplace = True)
                besio.append(df[columns[idx+2]][:l].values)
                df[columns[idx+3]].replace(np.nan, '', inplace = True)
                mol.append(df[columns[idx+3]][:l].values)
                df[columns[idx+4]].replace(np.nan, '', inplace = True)
                IorO.append(df[columns[idx+4]][:l].values)

    i = 0
    label_values = []
    while i < len(besio):
        label_values.append([])
        for j in range(len(besio[i])):
            #Strip unintentional whitespace from all entries:
            besio[i][j] = besio[i][j].replace(" ", "")
            mol[i][j] = mol[i][j].replace(" ", "")
            IorO[i][j] = IorO[i][j].replace(" ", "")
            
            if besio[i][j].upper() == 'O':
                label_values[i].append(besio[i][j].upper())
            if mol[i][j].upper() == 'PRO':
                label_values[i].append(besio[i][j].upper()+'-'+mol[i][j].upper())
            if IorO[i][j].upper() == 'I' or IorO[i][j].upper() == 'O':
                #The below does not handle cases where BESIO or MOL has errors though...
                if mol[i][j].upper() != 'PRO':
                    label_values[i].append(besio[i][j].upper()+'-'+mol[i][j].upper()+'-'+IorO[i][j].upper())
                else: 
                    print("Weird. This Property is organic or inorganic? LOL")
            #PRIME OPPORTUNITY FOR ERROR HANDLING - IF ANYTHING NOT IN THE ABOVE CATS, SOMETHING IS WRONG
            
        i += 1   
    return all_tokens, label_values

In [423]:
print(labels[6])

['O', 'O', 'O', 'O', 'S-PRO', 'S-PRO', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-MOL-O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-MOL-I', 'O', 'S-MOL-O', 'O', 'O', 'S-MOL-O', 'B-MOL-O', 'E-MOL-O', 'B-MOL-O', 'E-MOL-O', 'O', 'B-MOL-O', 'E-MOL-O', 'O', 'O', 'O', 'O', 'B-PRO', 'E-PRO', 'O', 'O', 'S-PRO', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-MOL-I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-PRO', 'O', 'S-MOL-I', 'O', 'S-MOL-O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-PRO', 'O', 'S-PRO', 'O', 'O', 'O', 'O', 'S-MOL-O', 'S-PRO', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [419]:
tokens, labels = extract_xy_(df)

ioro
ioro
PRO
PRO
ioro
ioro
ioro
PRO
ioro
PRO
PRO
PRO
ioro
ioro
ioro
PRO
PRO
ioro
PRO
ioro
ioro
PRO
PRO
ioro
PRO
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
PRO
PRO
ioro
PRO
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
PRO
PRO
PRO
PRO
ioro
ioro
ioro
PRO
PRO
PRO
PRO
ioro
ioro
PRO
PRO
ioro
ioro
ioro
PRO
PRO
ioro
PRO
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
PRO
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
PRO
PRO
ioro
ioro
PRO
PRO
PRO
PRO
ioro
PRO
ioro
ioro
ioro
ioro
ioro
ioro
PRO
PRO
PRO
PRO
PRO
PRO
ioro
ioro
PRO
PRO
PRO
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
PRO
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
PRO
PRO
PRO
PRO
PRO
PRO
PRO
ioro
ioro
ioro
PRO
PRO
ioro
ioro
ioro
ioro


Ok, so now we have two lists. Both lists are structured so that each entry represents a unique paper (ie, tokens[1] is a whole paper). We need to now chop it down so that all the extra entries at the end are removed, and the list tokens[1] is only as long as there are words in that paper. 

Once we have done that for each entry, we will need to build a looping/wrapper function that will read every excel sheet in a directory. It'd be ideal if that looping function would append each new list to the previous list, so we could end up with every single labeled paper in a single set of two lists. 

After we have that function built, the next step is to try to regenerate our sentence-split structure. First step is to make each paper back into a single string. We'll do this by making a homemade inverse .split() function, which means we'll add each item in the tokens[1] list together with a single whitespace between them. Example is in the case ['The', 'dog', 'ran.'] we would want to regen the original string of ['The dog ran.']. We could do that by doing original_string += (token[1][i]+ ' '). Once we have made each paper in the list back into individual strings, we'll chop each string into individual sentences by using NLTK. 

All of the above is now done!

From that point, it's more standard BERT. We'll use BERT's tokenizer. We'll need to make sure we hand-extend each label to match the tokenization done by the BERT tokenizer so we don't have length mismatches (a la: https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/ function tokenize_and_preserve_labels) and then we'll send it to a dataloader. 

In [5]:
#Ok, need another def function that takes in a list of tokens, and a list of labels, and returns them each as a list of strings
def tokenized_to_string(token_list):
    token_stringlist = []
    for paper_tokens in token_list:
        paper_string = ""
        for i in paper_tokens:
            #This is basically an 'unsplit' method lol
            paper_string += (str(i) + " ")
        token_stringlist.append(paper_string)
    return token_stringlist

In [6]:
def labeled_sheets_to_listed_tokens(directory_url):
    """This function opens a directory of labeled excel sheets from David's excel sheets and returns the tokens as a list 
    of strings fully combined on a document level. It returns a list of strings, with each string being a full document."""
    files = os.listdir(directory_url)
    token_list = []
    label_list = []
    sent_labels = []
    for file in files:
        df = pd.read_excel(directory_url+file)
        token, label = extract_xy(df)
        token_list += (tokenized_to_string(token))
        label_list += (label)
    #Now we tokenize each paper by sentences using NLTK:
    #we will also restructure labels to be ordered by sentences. 
    for i in range(len(token_list)):
        sentences = tokenize.sent_tokenize(token_list[i])
        token_list[i] = sentences
        short_term_labels = []
        for j in range(len(token_list[i])):    
            length = len(token_list[i][j].split())
            short_list = label_list[i][:length]
            short_term_labels.append(short_list)
            del(label_list[i][:length])
        sent_labels.append(short_term_labels)
    return token_list, sent_labels

In [14]:
dir_url = '/Users/Jonathan/Desktop/LabeledChemEData/Labeled_Sheets/'
list_o_tokens, list_o_labels = labeled_sheets_to_listed_tokens(dir_url)

Weird. This Property is organic or inorganic? LOL
Weird. This Property is organic or inorganic? LOL


In [472]:
print(list_o_tokens[5])
print(list_o_labels[5])

['In this work we study the carbonization resulting from an intumescence phenomenon of the fire-retardant formulation: ethylene terpolymer-ammonium polyphosphate/pentaerythritol.', 'Characterisation of the intumescent coating is carried out using infrared spectroscopy and MAS-NMR of the solid state.', 'The study shows the formation of organic phosphocarbonaceous esters, which limits depolymerisation or the evolution of gaseous hydrocarbons.', 'These esters prevent the development of condensed polyaromatic structures.', 'These latter are bridged by the polyethylenic links, which provide the mechanical properties of interest in the protective shield.', '©']
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-PRO', 'O', 'B-MOL-O', 'I-MOL-O', 'E-MOL-O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MOL-O', 'E-MOL-O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O',

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [10]:
def max_encoded_length(token_list):
    max_len = 0
    for papers in token_list:
        for sentences in papers:
            input_ids = tokenizer.encode(sentences, add_special_characters = True)
            max_len = max(max_len, len(input_ids))
            
    return max_len

Ok, so the longer the dataset input, the worse performance is going to get. It turns out the performance of BERT scales quadratically with feature size. Esp. when we only have like 17 sentences greater than 150 words, it seems insane to use so much extra dimensionality. Instead, what we'll do is we'll chunk up sentences to a specific length. BUT, in order to get context, we need to have overlap of each chunked sequence, though it's not clear exactly how much overlap we will need, as this is kinda non-standard. My thought is if we limit max input length to 150, we could then chunk stuff up into 150-length inputs, wherein there are like 25 input overlaps on either side of the sentence.  

Alternatively, we could just delete any sentences that are greater than a certain size. To begin with, we will just delete sentences.

If we chop all sentences with greater than 150 words, we lose 1.4% of our sentences.
If we chop all sentences with greater than 160 words, 
we lose 1.24%
If we chop all with greater that 175, we lose 0.78%
If we chop with greater than 200, we lose 0.38% (5 sentences)


We'll use 150 at first, because we want to try to train locally if at all possible.

In [11]:
def delete_sentences(token_list, label_list, print_pop = False, max_length = 150):
    i = 0
    #lets use pop to remove these long sentences
    while i < len(token_list):
        list_of_j = []
        j = 0
        while j < len(token_list[i]):
            input_ids = tokenizer.encode(token_list[i][j], add_special_tokens = True)
            if len(input_ids) > 150:
                print("Found item length: " + str(len(input_ids)))
                list_of_j.append(j)
            j += 1
        k = len(list_of_j)-1
        #Gotta count backwards so we don't disturb the list structure
        while k > -1:
            if print_pop:
                print(token_list[i].pop(list_of_j[k]))
                print(label_list[i].pop(list_of_j[k]))
            else:
                token_list[i].pop(list_of_j[k])
                label_list[i].pop(list_of_j[k])
            k = k - 1
        i += 1 
    return token_list, label_list

In [15]:
short_sent_list, short_label_list = delete_sentences(list_o_tokens, list_o_labels)

Found item length: 181
Found item length: 189
Found item length: 172
Found item length: 270
Found item length: 178
Found item length: 191
Found item length: 171
Found item length: 151
Found item length: 235
Found item length: 161
Found item length: 509
Found item length: 174
Found item length: 207
Found item length: 248
Found item length: 180
Found item length: 155
Found item length: 165
Found item length: 173


In [457]:
max_encoded_length(short_sent_list)

149

Because of this, we'll set our maximum length inside of BERT to be 155, just to be safe. It's probably a bit unnecessary, but for our relatively small dataset, I bet the extra 6D won't be a crisis.

Here's what I just learned: We can use the tensordataset loader things. We need to change our label list though. The label needs to have an O token for the CLS and SEP tokens, and that O token should also exist for the padding values. We also DO need to increase the length of each individual word label across the tokenized expansion of the word. 

Finally, once we've done all that, we need to convert the whole label list into individual integer mappings. We should build a label to number mapper, and then a number to label mapper at the same time. Should mostly just be a dictionary and key mapper type thing. 

In [16]:
#Long term we should absolutely consider getting rid of S, E labels
label_mapping = {'O': 0, "B-MOL-O": 1, "I-MOL-O": 2, "E-MOL-O": 3,
                "S-MOL-O": 4, "B-MOL-I": 5, "I-MOL-I": 6, "E-MOL-I": 7,
                "S-MOL-I": 8, "B-PRO": 9, "I-PRO": 10, "E-PRO": 11, "S-PRO": 12}


In [18]:
input_ids = []
attention_masks = []
label_shapes = []
no_pad_labels = []
count = 0

for abstracts, abst_labels in zip(short_sent_list, short_label_list):
    for sentences, sent_labels in zip(abstracts, abst_labels):
        encoded_dict = tokenizer.encode_plus(
                                    sentences,
                                    add_special_tokens = True,
                                    max_length = 155,
                                    pad_to_max_length = True,
                                    return_attention_mask = True,
                                    return_tensors = 'pt'
        )
        #Ok, now we get our labels based on encoded sizes. 
        #Make this a standalone function later instead of nesting
        #Need to start the CLS token to every label. 
        #This CLS token should be an 'O', to keep label
        #length matching consistent with the tokenized sentence
        extend_sent_labels = ['O']
        
        
        #I Bet the problem with things being read in is the difference in length of 
        #however this chunks sentences versus how the labels were originally split.
        
        for word, label in zip(sentences, sent_labels):
            tokenized_word = tokenizer.tokenize(word)
            #Find out how many chunks each word gets broken into
            n_subwords = len(tokenized_word)
            #Extend the length of the labels to match new word length
            #Put label in brackets so it knows you want n_subwords entries
            #of label, not label times n_subwords
            extend_sent_labels.extend([label]*n_subwords)
        
        #In order to know just how much to bias the dataset for each label, we need to know how many of each we have. 
        no_pad_labels.extend(extend_sent_labels)
        
        #This handles increasing the length for padding and sep tokens
        #Go all the way to 155. Padding and  SEP should both be O
        extend_sent_labels.extend(['O']*(155-len(extend_sent_labels)))    
#
        #Next step, we need to use the dictionary lookup
        #to replace all the values from this list to become 
        #numbers. for loops to go through the whole list. 
        #print(sentences)
        #print(extend_sent_labels)
        for i in range(len(extend_sent_labels)):
            if extend_sent_labels[i] in label_mapping.keys():
                #Replace the label in extend_set_labels[i] from dict
                extend_sent_labels[i] = label_mapping[extend_sent_labels[i]]
        #print(extend_sent_labels)
        #Then, we make the labels list into a tensor.
        #extend_sent_labels = torch.tensor(extend_sent_labels)
        test_list = []
        test_list.append(extend_sent_labels)
        test_list = torch.tensor(test_list)
        #Build our attention mask, labels, and input ids of each item.
        #print(encoded_dict)
        label_shapes.append(test_list)
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        #print(encoded_dict['input_ids'])
#         count += 1
#         if count == 2:
#             print(label_shapes)
#             print(input_ids)
#             raise TypeError
print(len(label_shapes[1]))
print(len(input_ids[1]))
print(len(attention_masks[1]))

#Make lists we just built into tensors
input_ids = torch.cat(input_ids, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.cat(label_shapes, dim = 0)

print("Original Sentence: ", short_sent_list[0][0])
print("Tokenized IDs: ", input_ids[0])
print(labels[0])

1
1
1
Original Sentence:  In the interaction between gas molecules with single-walled carbon nanotube (SWCNT) we show that as a result of collisions the gas scattering contributes with an important background signal and should be considered in SWCNT-based gas sensors.
Tokenized IDs:  tensor([  101,  1999,  1996,  8290,  2090,  3806, 10737,  2007,  2309,  1011,
        17692,  6351, 28991, 28251,  2063,  1006, 25430,  2278,  3372,  1007,
         2057,  2265,  2008,  2004,  1037,  2765,  1997, 28820,  1996,  3806,
        17501, 16605,  2007,  2019,  2590,  4281,  4742,  1998,  2323,  2022,
         2641,  1999, 25430,  2278,  3372,  1011,  2241,  3806, 13907,  1012,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0

In [21]:
#OK, now that we have a list of all labels, we need to convert them to their number forms
#From the number forms, we'll then build the relative weight for each of these entries. 

#THIS SHOULD BE A FUNCTION SINCE I CALL IT TWICE:
for i in range(len(no_pad_labels)):
    if no_pad_labels[i] in label_mapping.keys():
        #Replace the label in extend_set_labels[i] from dict
        no_pad_labels[i] = label_mapping[no_pad_labels[i]]

[0, 0, 0, 0, 0, 0, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 8, 0, 0, 0, 0, 0, 9, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 9, 11, 0, 11, 0, 0, 9, 10, 10, 10, 12, 0, 0, 0, 0, 9, 11, 0, 12, 12, 0, 12, 0, 0, 8, 0, 12, 0, 0, 0, 4, 4, 1, 3, 4, 4, 4, 4, 4, 1, 2, 3, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 12, 12, 0, 0, 0, 12, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 7, 0, 0, 0, 5, 6, 7, 8, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 5, 7, 0, 0, 0, 0, 0, 7, 0, 5, 7, 0, 0, 9, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 7, 0, 0, 8, 0, 0, 0, 0, 

In [25]:
#Now, we take the number converted label, and use it to build a list. 
#Initialize a list of zeros to count every instance of each key:
key_count_list = np.zeros(len(label_mapping.keys()))
for i in range(len(no_pad_labels)):
    key_count_list[no_pad_labels[i]] += 1

In [27]:
print(key_count_list)

[20731.   280.    72.   280.  1192.   157.    38.   162.   723.   248.
    77.   229.   433.]


In [28]:
sum(key_count_list)

24622.0

In [29]:
#Real quick percentage functions for each.
perc_list = []
for i in range(len(key_count_list)):
    perc_list.append(key_count_list[i]/sum(key_count_list))

In [31]:
print(perc_list)

[0.8419705954024855, 0.011371943790106408, 0.002924214117455934, 0.011371943790106408, 0.04841198927788157, 0.006376411339452522, 0.0015433352286572985, 0.006579481764275851, 0.029363983429453334, 0.010072293071237104, 0.0031272845422792624, 0.009300625456908455, 0.01758589878970027]


In [ ]:
#Ok, no we need to take our relative ratios of each, and use that to make biases. 
#The way to go from counts/percentages to rough initialization biases is:
#

In [501]:
attention_masks.shape

torch.Size([1272, 155])

In [502]:
labels.shape

torch.Size([1272, 155])

In [503]:
input_ids.shape

torch.Size([1272, 155])

Ok. Now we have a label list, an attention mask, and an input id that are all the same shape, and contain all the data, in full integer form, so everything can be put into a DataLoader, and then we can start thinking about putting everything into the BERT model. Very exciting! Let's start with a dataloader.

In [504]:
from torch.utils.data import TensorDataset, random_split

In [507]:
dataset = TensorDataset(input_ids, attention_masks, labels)

#Do our 90/10 training/validation split
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

#Now do a train_val split, randomly
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

1,144 training samples
  128 validation samples


In [508]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [509]:
#Set a dataloader batch size. Paper says do 16 or 32, but do bigger if comp can
batch_size = 32

#Create training dataloaders using a random sequence pull
train_dataloader = DataLoader(train_dataset,
                             sampler = RandomSampler(train_dataset),
                             batch_size = batch_size
                             )

#Validation can go any order, so we'll do sequentially
validation_dataloader = DataLoader(val_dataset,
                                  sampler = SequentialSampler(val_dataset),
                                  batch_size = batch_size
                                  )


In [33]:
from transformers import BertForTokenClassification, AdamW, BertConfig

In [35]:
label_numbers = len(label_mapping.keys())

In [58]:
model = BertForTokenClassification.from_pretrained(
        "bert-base-uncased",
        num_labels = label_numbers,
        output_attentions = False, #Whether model returns attention weights
        output_hidden_states = False, #Whether model outputs all hidden states
)

In [59]:
model.classifier.bias

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)

In [60]:
#OK, I need to make a torch.nn.Parameter out of my bias list, and then I can pass it to the classifier. 
#Some details: Since the default requires_grad for model is true, we'll keep that. 
#I still don't know how to calc the correct default biases, but we'll sort that out. I'll use perc_list for now 
#This should strictly make the model perform worse lol
#First, we need to make a tensor out of the per_list
perc_list_tensor = torch.tensor(perc_list)

new_classifier_bias = torch.nn.Parameter(perc_list_tensor, requires_grad = True)
print(new_classifier_bias)

Parameter containing:
tensor([0.8420, 0.0114, 0.0029, 0.0114, 0.0484, 0.0064, 0.0015, 0.0066, 0.0294,
        0.0101, 0.0031, 0.0093, 0.0176], dtype=torch.float64,
       requires_grad=True)


In [61]:
#Ok, now, we just set the new classifier bias to be equal to that and we're good to go~
model.classifier.bias = new_classifier_bias

In [62]:
model.classifier.bias

Parameter containing:
tensor([0.8420, 0.0114, 0.0029, 0.0114, 0.0484, 0.0064, 0.0015, 0.0066, 0.0294,
        0.0101, 0.0031, 0.0093, 0.0176], dtype=torch.float64,
       requires_grad=True)

In [54]:
#model.classifier.bias = perc_list

TypeError: cannot assign 'list' as parameter 'bias' (torch.nn.Parameter or None expected)

In [55]:
#model.classifier.bias = None

In [57]:
#print(model.classifier.bias)

None


In [ ]:
#model.named_parameters()

In [37]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

The BERT model has 201 different named parameters.



In [48]:
print(params[-1][0])

classifier.bias


In [49]:
print(params[-1][1])

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)


In [41]:
print(params[-2])

('classifier.weight', Parameter containing:
tensor([[ 0.0417,  0.0022,  0.0198,  ..., -0.0140, -0.0161, -0.0122],
        [ 0.0173, -0.0088, -0.0033,  ..., -0.0014,  0.0194,  0.0048],
        [-0.0078, -0.0309,  0.0159,  ..., -0.0197,  0.0299, -0.0307],
        ...,
        [ 0.0154, -0.0061, -0.0070,  ...,  0.0343,  0.0054,  0.0221],
        [-0.0200, -0.0007,  0.0093,  ...,  0.0205,  0.0308,  0.0139],
        [ 0.0131, -0.0187,  0.0436,  ...,  0.0221,  0.0187, -0.0014]],
       requires_grad=True))


In [40]:
#Print the parameters related to the classifier. 
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

bert.pooler.dense.weight                                  (768, 768)
bert.pooler.dense.bias                                        (768,)
classifier.weight                                          (13, 768)
classifier.bias                                                (13,)


In [515]:
#Need to run this on the cpu for right now :(
model.cpu()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

For the purposes of fine-tuning, the authors (ie Google AI) recommend choosing from the following values (from Appendix A.3 of the BERT paper):

Batch size: 16, 32 Learning rate (Adam): 5e-5, 3e-5, 2e-5 Number of epochs: 2, 3, 4 We chose:

Batch size: 32 (set when creating our DataLoaders) Learning rate: 2e-5 Epochs: 4 (we'll see that this is probably too many...)

The epsilon parameter eps = 1e-8 is "a very small number to prevent any division by zero in the implementation

In [518]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [516]:
from transformers import get_linear_schedule_with_warmup

In [519]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 1. Largely cause I'm not sure my computer will survive
epochs = 1

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [520]:
# Function to calculate the accuracy of our predictions vs labels
#This function is wrong, given my data format. 

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [521]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [522]:
device = torch.device("cpu")

In [527]:
import random
import time
import datetime

QUESTION: Did we mess up? Should we have everything set up a specific way to classify at a WORD level, rather than at a sentence level? I know that the below was for sentence level classification, and maybe the way it's set up to read in stuff is causing it to have issues with the way it is classifying stuff? Because in short, we should have way more samples than we actually do, I think? Like, if each word is an example, then we should have more batches.

I'm pretty confident this is the problem. I really do need to put together a simpler example. Maybe typecast all labels to just be binary. 1 if we care about it, 0 if we don't. That'd deal with the BESIO issue, and makes it so we need less training examples. We could then blink to 3 tags, 0 for outside, 1 for mol compounds, and 2 for properties. Then we can eventually get this whole thing here. 

Ok, so I looked into this. The doesn't seem to be correct. I've checked it against that "Depend on the definition" website NER strategy and it looks like the models for that one and this sentence classification one are identical. IDK why I'm getting zero accuracy. I think the next step is to go ahead and make some sort of probing function to see what is happening inside the NN. 

In [546]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 3 batches.
        if step % 3 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    #Gotta get these to build a confusion matrix.
    predictions, true_labels = [], []

    
    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU - Do we need?
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        #Need to keep track of predictions and true labels for conf matrix
        #First, we need to modify logits. IDK Why
        #I think we need to do it on 2. Logits structure goes in 3D. D1 is the list of all the sentences
        #D2 is the list of every token within each sentence. D3 is the label predicted values.
        new_preds = np.argmax(logits, axis=1).flatten()
        predictions.extend(new_preds)
        true_labels.extend(label_ids)        
        
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...
  Batch     3  of     36.    Elapsed: 0:02:15.
  Batch     6  of     36.    Elapsed: 0:04:25.
  Batch     9  of     36.    Elapsed: 0:06:34.
  Batch    12  of     36.    Elapsed: 0:08:45.
  Batch    15  of     36.    Elapsed: 0:10:17.
  Batch    18  of     36.    Elapsed: 0:12:18.
  Batch    21  of     36.    Elapsed: 0:13:57.
  Batch    24  of     36.    Elapsed: 0:15:43.
  Batch    27  of     36.    Elapsed: 0:17:39.
  Batch    30  of     36.    Elapsed: 0:19:33.
  Batch    33  of     36.    Elapsed: 0:21:26.

  Average training loss: 0.44
  Training epcoh took: 0:23:11

Running Validation...


C:\Users\Jonathan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """


  Accuracy: 0.00
  Validation Loss: 0.43
  Validation took: 0:00:59

Training complete!
Total training took 0:24:10 (h:mm:ss)


Next step after this is to obviously tune the model for our specific stuff, if this results is weird or whatever.
We'll need to do a hyperparameter optimization step.
We'll need to do a train and test split, instead of just a validation split. 
We'll need to find a way to save the model's final state. 
We need a way to use our model to predict the label for input documents. Right now, Idk how that would work. We should build it for a single sentence at a time. 

In [580]:
#What does flattening do to these?
print((new_preds))


[ 11   1  40  11   4  56  37 151   4  59 128  59   6   3   1  30   3  27
 123 111 145   3  28  49   3  80   0   1  11   1 119  72  96  12   9 102
 102  21  14   0   0  62   0  21  62  20 151  26  27 136  30  11   0   0
  77   0  77  77  64  77  17  27 105  31  15   0   1  39  10   4  13  64
  94 121  59 128  12   5   0  80  46   2 115  48  50  79  71  47 102  80
  61   0   3   5   0   4  94  31  96   2   5 136  18  10   0  19  95   0
  15  23  52 112  18   4  74  23  12   0  18  74   0  75 102  90 104  18
  25  60  27  21   3   1  93   8   4  13  58  93  11  59  39  11   5   0
  19   5   0   8  13  10  10  19  15 102  22  10  21 133  22   0  84  25
  21  21   1  19   0  25  12  40   8  10  45  41  55  46  46  47  35  55
  54  38   0  18  38   0 114 127  44   5  20  26  49  48  41   0   1  80
   1  92  78  75  16  34   3  79  38  17   0   3  13   0  75  82  23  11
  43  38  75  49  19   0  68  40   0  16  91  68  94  16  41  94  18   4
   2  16 154   0   8  94  58   5  21  13  94  21  2

In [605]:
logits[0][0]

array([ 5.2491117 , -0.7033895 , -0.92590857, -0.01007925,  0.585069  ,
       -0.47741717, -0.79565704, -1.0148675 , -0.15901805, -0.5961771 ,
       -0.6242501 , -0.6919883 , -0.36357838], dtype=float32)

In [617]:
logits[0]

array([[ 5.2491117 , -0.7033895 , -0.92590857, ..., -0.6242501 ,
        -0.6919883 , -0.36357838],
       [ 5.070979  , -0.51532876, -1.0048454 , ..., -0.77022284,
        -0.72542137, -0.43829173],
       [ 5.124782  , -0.7417729 , -0.92641526, ..., -0.7831685 ,
        -0.68256766, -0.2600086 ],
       ...,
       [ 4.4302125 , -0.6459397 , -1.0174502 , ..., -0.6583927 ,
        -0.6276074 , -0.28761747],
       [ 4.437419  , -0.6412594 , -0.98670673, ..., -0.65824443,
        -0.6325286 , -0.2791918 ],
       [ 4.452333  , -0.67401826, -0.9768401 , ..., -0.61315876,
        -0.6335059 , -0.28955418]], dtype=float32)

In [616]:
max_list = []
argmax_list = []
for i in range(len(logits[0])):
    #print(logits[0][i])
    max_list.append(logits[0][i].max())
    argmax_list.append(np.argmax(logits[0][i]))
print(max_list)
print(argmax_list)

[5.2491117, 5.070979, 5.124782, 5.288239, 5.2486434, 5.1727395, 5.1886578, 5.10943, 5.2577887, 5.213809, 5.2581344, 5.296952, 5.207807, 5.2183857, 4.5887866, 4.818345, 4.5120826, 4.946184, 4.5471864, 4.565126, 4.545998, 4.4641733, 4.5813403, 4.6074324, 4.590307, 4.578827, 4.8047137, 4.555406, 4.8236785, 4.4607663, 4.508288, 4.501936, 4.4208603, 4.50302, 4.4620614, 4.481033, 4.490369, 4.477613, 4.4781156, 4.68349, 4.5388007, 4.577471, 5.0233145, 4.631639, 4.48982, 4.5687675, 4.595466, 4.7474003, 4.493668, 4.551702, 4.563682, 4.5457377, 4.821492, 4.966087, 4.578956, 4.539617, 4.449919, 4.473865, 4.4408956, 4.4199233, 4.375855, 4.4480066, 4.447887, 4.4497, 4.474768, 4.4551454, 4.993584, 5.0306315, 4.9739213, 4.9985204, 4.587377, 4.872846, 5.012733, 4.60172, 4.499279, 4.5526204, 4.5495334, 4.6393695, 4.559115, 4.603374, 4.7455945, 4.5001082, 4.9851756, 4.870842, 4.5508437, 4.5215354, 4.422343, 4.491991, 4.4015102, 4.3817763, 4.439386, 4.451861, 4.424936, 4.4918704, 4.507355, 4.531614, 5.02

In [593]:
faek = np.argmax(logits, axis=0)

In [595]:
faek

array([[ 5,  3,  1, ...,  1, 19, 30],
       [18, 30,  7, ...,  1,  1, 14],
       [ 5, 14,  1, ...,  1, 20, 30],
       ...,
       [28, 30,  1, ...,  1,  1,  3],
       [29, 30,  1, ...,  1,  1, 30],
       [20, 30,  1, ...,  1,  1, 30]], dtype=int64)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [567]:
print(len(logits))

32


In [529]:
from sklearn.metrics import confusion_matrix

In [551]:
def annotated_confusion_matrix(predictions, true_labels, label_dict):
    """
    Create an annotated confusion matrix by adding label
    annotations and formatting to sklearn's `confusion_matrix`.
    """

    # Create header from unique tags
    header = sorted(label_dict.values())

    # Calculate the actual confusion matrix
    matrix = confusion_matrix(true_labels, predictions, labels=header)

    # Final formatting touches for the string output
    mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(matrix)]
    content = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

    return content

In [563]:
print(len(predictions))

1664


In [562]:
len(true_labels)

128

In [552]:
annotated_confusion_matrix(predictions, true_labels, label_mapping)

ValueError: Found input variables with inconsistent numbers of samples: [128, 1664]

In [535]:
outputs = np.argmax(logits, axis=1).flatten()

In [545]:
outputs

(416,)

In [543]:
label_ids[15]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        9, 11,  0,  2,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0], dtype=int64)

In [537]:
len(outputs)

416

In [536]:
print(outputs)

[ 11   1  40  11   4  56  37 151   4  59 128  59   6   3   1  30   3  27
 123 111 145   3  28  49   3  80   0   1  11   1 119  72  96  12   9 102
 102  21  14   0   0  62   0  21  62  20 151  26  27 136  30  11   0   0
  77   0  77  77  64  77  17  27 105  31  15   0   1  39  10   4  13  64
  94 121  59 128  12   5   0  80  46   2 115  48  50  79  71  47 102  80
  61   0   3   5   0   4  94  31  96   2   5 136  18  10   0  19  95   0
  15  23  52 112  18   4  74  23  12   0  18  74   0  75 102  90 104  18
  25  60  27  21   3   1  93   8   4  13  58  93  11  59  39  11   5   0
  19   5   0   8  13  10  10  19  15 102  22  10  21 133  22   0  84  25
  21  21   1  19   0  25  12  40   8  10  45  41  55  46  46  47  35  55
  54  38   0  18  38   0 114 127  44   5  20  26  49  48  41   0   1  80
   1  92  78  75  16  34   3  79  38  17   0   3  13   0  75  82  23  11
  43  38  75  49  19   0  68  40   0  16  91  68  94  16  41  94  18   4
   2  16 154   0   8  94  58   5  21  13  94  21  2

In [533]:
print(label_ids)

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  9 11 ...  0  0  0]]


In [532]:
#Build a confusion matrix!
con_matrix = confusion_matrix(y_true = label_ids, y_pred = logits, 
                              labels = label_mapping.values())

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and unknown targets